In [1]:
import sys
sys.path.append("..")

from qiskit.providers.fake_provider import Fake27QPulseV1, Fake5QV1, Fake20QV1,Fake127QPulseV1,GenericBackendV2
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from src.state_of_the_art.sabre import run_sabre
from src.state_of_the_art.cirq import run_cirq,Sycamore
from src.state_of_the_art.pytket import run_pyket
from src.state_of_the_art.qmap import run_qmap
from src.state_of_the_art.bqskit import run_bqskit
from src.isl_routing.mapping.routing import POLY_QMAP
from src.isl_routing.utils.isl_data_loader import *
from src.isl_routing.utils.circuit_utils import *
from src.visiualisation.plots import *

from time import time

import warnings
warnings.filterwarnings("ignore")


In [5]:
edges = FakeSherbrooke().configuration().coupling_map
num_qubits = FakeSherbrooke().configuration().n_qubits
#edges = Sycamore()


#backend = load_backend("../topologies/Ankaa-3.json")
#edges = backend['coupling_map']

In [3]:
file_path = f"../benchmarks/polyhedral/queko-bss-81qbt/81QBT_100CYC_QSE_0.json"
data = json_file_to_isl(file_path)

In [6]:
poly_mapper = POLY_QMAP(edges, data)
poly_mapper.run(heuristic_method="closure",verbose=1,initial_mapping_method="trivial")
poly_mapper.results[0]

{'swap_count': 3731, 'circuit_depth': 864}

In [5]:
poly_mapper.run(heuristic_method="closure",verbose=1,no_read_dep=True,initial_mapping_method="trivial")
poly_mapper.results[0]

{'swap_count': 3731, 'circuit_depth': 864}

In [4]:
run_sabre(data,edges,layout="trivial")

{'swaps': 48, 'depth': 56, 'cx_count': 0}

In [5]:
run_cirq(data,edges,initial_mapping="trivial")

{'swaps': 43, 'depth': 42, 'cx_count': 45}

In [6]:
poly_mapper = POLY_QMAP(edges, data)
poly_mapper.run(heuristic_method="closure",verbose=1,initial_mapping_method="trivial")
poly_mapper.results[0]

{'swap_count': 40, 'circuit_depth': 46}

In [27]:
run_pyket(data,edges,initial_mapping="trivial")

In [7]:
run_qmap(data,edges,initial_mapping="trivial")

{'swaps': 42, 'depth': 80, 'cx_count': 45}

In [4]:
run_qmap(data,edges,initial_mapping="trivial")
poly_mapper = POLY_QMAP(edges, data)
poly_mapper.run(heuristic_method="closure",verbose=1,initial_mapping_method="trivial")
print(poly_mapper.results[0])
run_cirq(data,edges,initial_mapping="trivial")
run_sabre(data,edges,layout="trivial")

In [5]:
import tempfile
from bqskit import Circuit, compile, MachineModel
from bqskit.ir.gates import SwapGate,CNOTGate, CNOTGate, U3Gate, SwapGate,XGate
from bqskit.compiler import GateSet,BasePass,PassData


In [7]:
qasm_str = data["qasm_code"]
with tempfile.NamedTemporaryFile(mode='w', suffix='.qasm', delete=True) as tmp:
    tmp.write(qasm_str)
    tmp.flush()  
    circuit = Circuit.from_file(tmp.name)

edges = [tuple(edge) for edge in edges]  
gate_set = GateSet([CNOTGate(), XGate(),U3Gate(), SwapGate()])
machine = MachineModel(num_qubits, coupling_graph=edges, gate_set=gate_set)


In [ ]:
compiled_circuit = compile(
        circuit,
        machine,
        seed=42,
        optimization_level=3,
    )

In [9]:
from collections import Counter
gate_counts = Counter(op.gate.name for op in circuit)
print(gate_counts)

gate_counts = Counter(op.gate.name for op in compiled_circuit)
print(gate_counts)

Counter({'XGate': 816, 'CNOTGate': 320})
Counter({'U3Gate': 1445, 'CNOTGate': 254, 'SwapGate': 202})


{'swaps': 179, 'depth': 300, 'cx_count': 320}

In [6]:
run_qmap(data,edges)

{'swaps': 125, 'depth': 314}

In [12]:
run_cirq(data,edges,initial_mapping="trivial")

{'swaps': 156, 'depth': 366, 'cx_count': 320}

In [13]:
run_qmap(data,edges,initial_mapping="trivial")

{'swaps': 230, 'depth': 279, 'cx_count': 320}

In [ ]:
run_bqskit(data,edges,num_qubits)

In [14]:
run_sabre(data,edges,layout="trivial")

{'swaps': 8097, 'depth': 4227, 'cx_count': 5400}

In [12]:
#qc.draw(output="mpl")

In [10]:
poly_mapper.circuit.depth()

298

In [11]:
from qiskit.transpiler import CouplingMap
 
def is_routable(circuit, coupling_map):
    cm = CouplingMap(coupling_map)
    for instr, qargs, _ in circuit.data:
        if instr.num_qubits > 1:
            q0 = qargs[0]._index
            q1 = qargs[1]._index
            if cm.distance(q0, q1) != 1:
                print(f"Gate {instr.name} on qubits {q0} and {q1} is not directly supported.")
                return False
 
        return True
 

routable = is_routable(poly_mapper.circuit, edges)
if routable:
    print("The circuit is directly routable on the backend.")
else:
    print("The circuit is NOT directly routable on the backend.")
    

The circuit is directly routable on the backend.


In [12]:
#poly_mapper.circuit.draw(output="mpl")

In [13]:
from qiskit.visualization import plot_gate_map
 
#plot_gate_map(FakeSherbrooke(),figsize=(8,8))



In [3]:
file_path = f"../benchmarks/polyhedral/queko-bss-16qbt/16QBT_100CYC_QSE_0.json"
data = json_file_to_isl(file_path)
qasm_str = data["qasm_code"]

In [14]:
from qiskit.circuit.library.basis_change.qft import QFT

qc = QFT(97,do_swaps=False).decompose()

In [15]:
qc.count_ops()

OrderedDict([('cp', 4656), ('h', 97)])

In [2]:
from qiskit.qasm2 import dump, dumps  # Import the new QASM export functions
from qiskit.circuit.library.basis_change.qft import QFT

for i in range(10,81):
    qc = QFT(i,do_swaps=False).decompose()
    with open(f'qft/{i}.qasm', 'w') as f:
        dump(qc, f)